# Introduction

Download financial data of the stocks in OMXSPI (390ish stocks listed in Stockholm) and rank them based on the financial metric EV / EBIT

IMPORTANT: I have implemented some measures to avoid rate limitations when downloading data. But, the block labelled 'ev / ebit calculation' will likely have to be run multiple times due to Yahoo Finance FREE API limiting access.

The partial results are saved as pkl files each time so it finishes eventually without having to do the same work multiple times.

January 2025.

# Download data.

In [ ]:
# download ticker data for OMXSPI from Nasdaq OMX

# example address:
# 'https://indexes.nasdaqomx.com/Index/ExportWeightings/OMXSPI?tradeDate=2025-01-16T00:00:00.000&timeOfDay=SOD'

# need to replace '2025-01-16' with todays date in yyyy-mm-dd format.
# this ensures that I always have the latest index-components.

from datetime import date

# Get today's date
today = date.today()
print("Today's date:", today)

# string1 is static part of the HTTP adress
# string 2 is dynamically updated.

string1 = 'https://indexes.nasdaqomx.com/Index/ExportWeightings/OMXSPI?tradeDate='
string2 = str(today) + 'T00:00:00.000&timeOfDay=SOD'
http = string1 + string2

# the '{http}' makes sure the variable http is being treated as a string.
# this make sure there are no problems with special characters.

!wget -O tickers.xlsx '{http}'

Today's date: 2025-04-25
--2025-04-25 06:58:19--  https://indexes.nasdaqomx.com/Index/ExportWeightings/OMXSPI?tradeDate=2025-04-25T00:00:00.000&timeOfDay=SOD
Resolving indexes.nasdaqomx.com (indexes.nasdaqomx.com)... 45.60.150.18
Connecting to indexes.nasdaqomx.com (indexes.nasdaqomx.com)|45.60.150.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21446 (21K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘tickers.xlsx’

tickers.xlsx        100%[===================>]  20.94K  --.-KB/s    in 0.04s   

2025-04-25 06:58:21 (547 KB/s) - ‘tickers.xlsx’ saved [21446/21446]



In [ ]:
import pandas as pd
import numpy as np

# Read the ticker file into a Pandas DataFrame
df = pd.read_excel("tickers.xlsx", engine="openpyxl")
df = df.dropna()

In [ ]:
tickers = tickers = df['Unnamed: 1'].iloc[1:]   #.iloc[1:] removes first row
tickers_l = tickers.tolist() # create list of tickets.
print(f' 5 first components of tickers_l: {tickers_l[:5]}')

 5 first components of tickers_l: ['8TRA', 'AAK', 'ABB', 'ACAD', 'ACE']


In [ ]:
# Go from the Security Symbols used by Nasdaq to yahoo finance tickers.
# this is necessary because the data is being fetched from Yahoo Finance.

def convert_to_yahoo_ticker(name):
    # Replace spaces with dashes and append .ST for Stockholm stocks
    ticker = name.replace(" ", "-") + ".ST"
    return ticker

# Example usage
#companies = ["XANO B", "VOLVO B", "ERIC B"]
#yahoo_tickers = [convert_to_yahoo_ticker(company) for company in companies]

In [ ]:
yahoo_tickers = [convert_to_yahoo_ticker(company) for company in tickers_l]
yahoo_tickers[:5]

['8TRA.ST', 'AAK.ST', 'ABB.ST', 'ACAD.ST', 'ACE.ST']

In [ ]:
# Remove the A entry if both A and B stocks are in the list.
# for Example, we don't need 'ACRI-A.ST', and 'ACRI-B.ST', get rid of A.

# A is usually less liquid so that's the one to get rid of.
# Some exceptions exist. SHB B is for example less liquid than
# SHB A. But, in general A is the one to get rid of.

# Set to keep track of the stocks we want to keep
tickers_list = set()

# Iterate over each ticker
for ticker in yahoo_tickers:
    # Check if it's a version A (contains '-A') and see if its B counterpart exists
    if '-A' in ticker:
        counterpart = ticker.replace('-A', '-B')
        if counterpart in yahoo_tickers:
            continue  # Skip the A version if the B version exists
    # Add to the set if it's not an A version or no B counterpart exists
    tickers_list.add(ticker)

# Convert the set back to a list (optional)
tickers_list = list(tickers_list)

print(tickers_list)
print(len(tickers_list))

# EV / EBIT calculation

In [ ]:
# This block: compute EV / EBIT for all companies in tickers_list.

# This code may need to be ran several times due to rate limitations.

# 2 second sleep is implemented in the loop to mitigate the risk of
# getting rate limited.

# Saving valuations as pickles is implemented to deal with the loop being
# broken by rate limitations.


import yfinance as yf
import pickle
import os
import time

# Create directory to save individual valuations
os.makedirs("valuations", exist_ok=True)

# List of potential debt fields
debt_fields = ['Net Debt', 'Total Debt', 'Total Liabilities Net Minority Interest']

valuations = {}

for ticker in tickers_list:
    # Skip if already saved
    file_path = f"valuations/{ticker}.pkl"
    if os.path.exists(file_path):
        print(f"Skipping {ticker} — already saved.")
        continue

    # Create the Ticker object
    company = yf.Ticker(ticker)

    # Try to access company.info
    sector = company.info.get('sector', None)

    # If company.info is missing or 'sector' is unavailable, skip this ticker
    if not sector:
        print(f"Skipping {ticker} due to missing 'sector' info.")
        continue

    # Exclude financials and RE.
    if sector == 'Financial Services' or sector == 'Real Estate':
        print('Skipping ticker ', ticker, 'sector: ', sector)
        continue

    # If the company is not in financials, perform EV / EBIT calculation
    try:
        MC = company.info['marketCap']
    except:
        print(f"Missing market cap for {ticker}")
        continue

    net_debt = None

    for debt_field in debt_fields:
        if debt_field == 'Net Debt':
            try:
                net_debt_series = company.balance_sheet.loc['Net Debt']
                net_debt = net_debt_series.iloc[0]
                if net_debt != net_debt:  # check for NaN
                    raise KeyError
                break
            except KeyError:
                continue
        else:
            try:
                total_debt_s = company.balance_sheet.loc[debt_field]
                total_debt = total_debt_s.iloc[0]
                cash_s = company.balance_sheet.loc['Cash And Cash Equivalents']
                cash = cash_s.iloc[0]
                net_debt = total_debt - cash
                break
            except KeyError:
                continue

    if net_debt is None:
        print(f"No valid debt data for {ticker}")
        continue

    ev = MC + net_debt

    try:
        ebit_series = company.income_stmt.loc['EBIT']
        ebit = ebit_series.iloc[0]

        ev_ebit = ev / ebit
        valuations[ticker] = ev_ebit

        # Save individual valuation
        with open(file_path, 'wb') as f:
            pickle.dump(ev_ebit, f)

        print(f"{ticker}: EV/EBIT = {ev_ebit:.2f} — saved.")

    except KeyError:
        print('No EBIT data for ticker: ', ticker)
        valuations[ticker] = -1

    # Delay to avoid rate limiting
    time.sleep(2)


In [ ]:
# load the valuations

import os
import pickle

valuations = {}  # reset or create new

# SET TO YOUR DIRECTORY WHERE 'valuations' WAS SAVED.
dataroot = '/content' # root directory in Google Colab

# path to the folder where each ticker's valuation as a separate .pkl
valuations_folder = os.path.join(dataroot, 'valuations')

# creation valuations dict.

for filename in os.listdir(valuations_folder):
    if filename.endswith(".pkl"):
        ticker = filename.replace(".pkl", "")
        filepath = os.path.join(valuations_folder, filename)
        with open(filepath, "rb") as f:
            valuation = pickle.load(f)
            valuations[ticker] = valuation


In [ ]:
# Filter out negative values and then sort in ascending order
# treat items as float to avoiding sorting strings (won't work)

import math

# Filter out NaN values and negative values.
# Negative valuations means the company is losing money, this is no good.
filtered_valuations = {k: v for k, v in valuations.items() if not math.isnan(v) and v > 0}

# Sort the dictionary by value
sorted_data = sorted(filtered_valuations.items(), key=lambda item: item[1])

# n number of companies to display
fraction = 0.10 # 0.10 --> top 10% of companies.
n = int(fraction * len(tickers))

# Sort the dictionary by the values (EV/EBIT) and then slice the top `n`
top_companies = sorted(sorted_data, key=lambda item: item[1])[:n]

In [ ]:
# creat ticker --> company name mapping

# Create dictionary from col1 as keys and col2 as values
mapping_dict = dict(zip(df['Unnamed: 1'].iloc[1:], df['Unnamed: 0'].iloc[1:]))

# convert to yahoo keys.

# Create a new dictionary with modified keys
yahoo_mapping_dict = {convert_to_yahoo_ticker(key): value for key, value in mapping_dict.items()}

In [ ]:
# Display the results.


print(f'Highest {n} (top {fraction*100} %) ranked non-financial and non-real-estate companies in OMXSPI by EV / EBIT: ', '\n')

for key, value in top_companies:
    print(yahoo_mapping_dict[key], ' EV / EBIT:', round(value, 2))

Highest 39 (top 10.0 %) ranked non-financial and non-real-estate companies in OMXSPI by EV / EBIT:  

Cinclus Pharma Holding AB  EV / EBIT: 0.52
Volvo Car AB ser. B  EV / EBIT: 1.49
Eniro Group AB  EV / EBIT: 2.23
SSAB AB ser. B  EV / EBIT: 2.49
Saniona AB  EV / EBIT: 2.74
Enea AB  EV / EBIT: 3.63
Bong AB  EV / EBIT: 5.09
Dedicare AB ser. B  EV / EBIT: 5.33
Scandic Hotels Group AB  EV / EBIT: 5.48
G5 Entertainment AB  EV / EBIT: 5.77
AcadeMedia AB  EV / EBIT: 5.94
Bulten AB  EV / EBIT: 6.38
ProfilGruppen AB ser. B  EV / EBIT: 6.41
NOVOTEK AB ser. B  EV / EBIT: 6.57
Nordic Paper Holding AB  EV / EBIT: 6.63
Boliden AB  EV / EBIT: 6.67
Arctic Paper S.A.  EV / EBIT: 6.77
B3 Consulting Group AB  EV / EBIT: 7.01
Pricer AB ser. B  EV / EBIT: 7.31
Profoto Holding AB  EV / EBIT: 7.39
Nilorngruppen AB Ser. B  EV / EBIT: 7.65
Green Landscaping Group AB  EV / EBIT: 7.81
Humana AB  EV / EBIT: 7.99
ATTENDO AB  EV / EBIT: 8.0
VBG GROUP AB ser. B  EV / EBIT: 8.38
Sleep Cycle AB  EV / EBIT: 8.56
SKF, A